In [2]:
from __future__ import print_function
import numpy as np
import bayesiancoresets as bc
from scipy.optimize import minimize
from scipy.linalg import solve_triangular
import time
import sys, os
import argparse
import pandas as pd
import pystan
import pickle

#make it so we can import models/etc from parent folder
sys.path.insert(1, os.path.join(sys.path[0], '../../examples/common'))
import results
import plotting
import radon

In [3]:
import importlib; importlib.reload(radon); 

In [4]:
data_dict_ = radon.load_data()

# load stanfit (or make one if no cache exists)
path_with_data = 'stancache/weighted_radon.pkl'
if os.path.isfile(path_with_data):
    sm = pickle.load(open(path_with_data, 'rb'))
else:
    sm = pystan.StanModel(model_code=radon.weighted_varying_intercept)
    with open(path_with_data, 'wb') as f: pickle.dump(sm, f)
        
path_without_data = 'stancache/radon_prior.pkl'
if os.path.isfile(path_without_data):
    sm_prior = pickle.load(open(path_without_data, 'rb'))
else:
    sm_prior = pystan.StanModel(model_code=radon.prior_code)
    with open(path_without_data, 'wb') as f: pickle.dump(sm_prior, f)

## load pystan fit object, which has functions to eval log_likelihood
## we don't really need the samples from this stanfit, but it's convenient
## to automatically evaluate log_likelihood (as opposed to making a separate
## function to do it)

stanfit = sm.sampling(data=data_dict_)
# print(stanfit)
priorfit = sm_prior.sampling(data=data_dict_)
# likelihood = lambda pts, th: stanfit.log_prob(stanfit.unconstrain_pars(th))

In [5]:
stanfit = sm.sampling(data=data_dict_, verbose=True)

## Compare prior and posterior samples

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1,2)

post_df = stanfit.to_dataframe()
pri_df = priorfit.to_dataframe()
var_name = "a[1]"

axes[0].hist(post_df[var_name])
axes[0].set_title('posterior %s' %var_name)

axes[1].hist(pri_df[var_name])
axes[1].set_title('prior %s' %var_name)

plt.tight_layout()
plt.show()

## Inspect the log likelihoods

In [ ]:
n_list = range(1,3)
for idx in n_list:
    varname = "ll[%d]" %idx
    plt.figure()
    plt.hist(post_df[varname])
    plt.show()
    plt.close()

In [ ]:
varnames = ["ll[%s]" %idx for idx in range(1,10)]
sub_df = post_df[varnames]
sub_df.head()

In [ ]:
np.array(sub_df).T